In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../data/adult23.csv", sep=",")

['URBRRL',
 'RATCAT_A',
 'INCTCFLG_A',
 'IMPINCFLG_A',
 'LANGSPECR_A',
 'LANGSOC_A',
 'LANGDOC_A',
 'LANGMED_A',
 'LANGHM_A',
 'PPSU',
 'PSTRAT',
 'HISPALLP_A',
 'RACEALLP_A',
 'DISAB3_A',
 'SCHDYMSSTC_A',
 'AFNOW',
 'REPWRKDYTC_A',
 'YRSINUS_A',
 'CITZNSTP_A',
 'PRTNREDUCP_A',
 'SPOUSEDUCP_A',
 'LEGMSTAT_A',
 'MARSTAT_A',
 'SASPPRACE_A',
 'SASPPHISP_A',
 'PRTNRAGETC_A',
 'SPOUSAGETC_A',
 'PRTNRWKFT_A',
 'PRTNRWRK_A',
 'SPOUSWKFT_A',
 'SPOUSWRK_A',
 'SPOUSESEX_A',
 'PRTNRSEX_A',
 'INJWRKDYTC_A',
 'NUMINJTC_A',
 'SHINGYEARP_A',
 'HHRESPSA_FLG',
 'PCNTADWFP1_A',
 'PCNTADWKP1_A',
 'FDSCAT4_A',
 'FDSCAT3_A',
 'EPINUMSEZP_A',
 'EMPDYSMSS3_A',
 'EMPLSTWOR1_A',
 'EMPWRKFT1_A',
 'EMPWRKLSW1_A',
 'EMPWKHRS3_A',
 'EMDOCCUPN2_A',
 'EMDOCCUPN1_A',
 'EMDINDSTN2_A',
 'EMDINDSTN1_A',
 'DIBAGETC_A',
 'DIFYRSTC1_A',
 'SMKECIGST_A',
 'SMKCIGST_A',
 'BMICAT_A',
 'WEIGHTLBTC_A',
 'HEIGHTTC_A',
 'URGNT12MTC_A',
 'EMERG12MTC_A',
 'DIBA1CNMT_A',
 'PCNT18UPTC',
 'PCNTLT18TC',
 'PHQ2SCREEN_A',
 'GAD2SCREEN_A',